In [ ]:
# find a list of suitable cities in North Carolina to have a headquarter/branch of any company based on its specific parameters.

In [1]:
# Dependencies Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as req
import json
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import interactive, Layout, Textarea, Dropdown, Label
HTML('<style> .widget-hbox .widget-label { max-width:350ex; text-align:left} </style>')

In [12]:
# Population Catagory Options in a list 
population_catagory = ['>=100000','>=250000','>=500000','>=1000000']

labor_force_catagory = ['Management','Business and Financial Operations','Computer and Mathematical',
                        'Architecture and Engineering','Life, Physical, and Social Science',
                        'Community and Social Service','Legal','Education, Training, and Library',
                        'Arts, Design, Entertainment, Sports, and Media','Healthcare Practitioners and Technical',
                        'Healthcare Support','Protective Service','Food Preparation and Serving Related',
                        'Building and Grounds Cleaning and Maintenance','Personal Care and Service',
                        'Sales and Related','Office and Administrative Support','Farming, Fishing, and Forestry',
                        'Construction and Extraction','Installation, Maintenance, and Repair','Production',
                        'Transportation and Material Moving', 'Information Technology', 'Data Science', 'Research', 'Pharmaceutical']

In [13]:
form_item_layout = Layout(
    display='flex',
    #flex_flow='row',
    justify_content='space-between',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='70%'
)

In [14]:
census = pd.read_csv('Census.csv', encoding="ISO-8859-1")
census.head()

,City,Population,County,State,FullState,MSA,Population Score,Closest Airport,Proximity to Airport,Airport Score
0,New York,8537673,Bronx County,NY,New York,"New York-Jersey City-White Plains, NY-NJ Metro...",10.0,Newark Liberty International Airport,28 min,0.8
1,Los Angeles,3976322,Los Angeles County,CA,California,"Los Angeles-Long Beach-Glendale, CA Metropolit...",10.0,Los Angeles International Airport,25 min,2.0
2,Chicago,2704958,Cook County,IL,Illinois,"Chicago-Naperville-Arlington Heights, IL Metro...",10.0,O'Hare International Airport,31 min,0.0
3,Houston,2303482,Harris County,TX,Texas,"Houston-The Woodlands-Sugar Land, TX",10.0,George Bush Intercontinental Airport,21 min,3.6
4,Phoenix,1615017,Maricopa County,AZ,Arizona,"Phoenix-Mesa-Scottsdale, AZ",10.0,Phoenix Sky Harbor International Airport,9 min,8.4


In [6]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
City                    303 non-null object
Population              303 non-null int64
County                  303 non-null object
State                   303 non-null object
FullState               303 non-null object
MSA                     303 non-null object
Population Score        303 non-null float64
Closest Airport         303 non-null object
Proximity to Airport    303 non-null object
Airport Score           303 non-null float64
dtypes: float64(2), int64(1), object(7)
memory usage: 23.8+ KB


In [7]:
laborforce = pd.read_csv('LaborForce.csv')
laborforce.head()

,AREA_NAME,OCC_TITLE,TOT_EMP,JOBS_1000,LOC QUOTIENT,Score
0,"Abilene, TX",Management,"2,030",31.741,0.63,1.3
1,"Abilene, TX",Business and Financial Operations,"2,230",34.804,0.67,1.7
2,"Abilene, TX",Computer and Mathematical,920,14.402,0.49,0.0
3,"Abilene, TX",Architecture and Engineering,710,11.068,0.62,1.2
4,"Abilene, TX","Life, Physical, and Social Science",380,5.901,0.72,2.2


In [15]:
# Define Population function to choose the population catagory as part of the Selection Widgets
def f(Population):
    if Population =='>=100000':
        Population = 100000
    elif Population == '>=250000':
        Population = 250000
    elif Population == '>=500000':
        Population = 500000
    else:
        Population = 1000000
    return Population

# Create Lists using interactive
Population = interactive(f,Population=population_catagory
                         ,layout=form_item_layout)

# You can also view this in a notebook without using display.
display(Population)

interactive(children=(Dropdown(description='Population', options=('>=100000', '>=250000', '>=500000', '>=10000…

In [18]:
# This is to show the user selection value for the Minimum Population 
Minimum_population = Population.result
Minimum_population

500000

In [19]:
# Define Labor force Catagory function to choose the Labor force Catagory as part of the Selection Widgets
def f(Primary):
    return Primary
# Create Lists using interactive
primary = interactive(f,Primary=labor_force_catagory,layout=form_item_layout)

# You can also view this in a notebook without using display.
display(primary)

interactive(children=(Dropdown(description='Primary', options=('Management', 'Business and Financial Operation…

In [21]:
Primary = primary.result
Primary

'Transportation and Material Moving'

In [22]:
Secondary_labor_force_catagory = [x for x in labor_force_catagory if x != Primary]


In [23]:
# Define Labor force Catagory function to choose the Labor force Catagory as part of the Selection Widgets
def f(Secondary):
    return Secondary
# Create Lists using interactive
secondary = interactive(f,Secondary=Secondary_labor_force_catagory,layout=form_item_layout)

# You can also view this in a notebook without using display.
display(secondary)

interactive(children=(Dropdown(description='Secondary', options=('Management', 'Business and Financial Operati…

In [ ]:
Secondary = secondary.result
Secondary

In [24]:
# This is to have an option for the user to choose the Optional Labor Force form the list of values other then Primary and Secondary
optional_labor_force_catagory = [x for x in Secondary_labor_force_catagory if x != secondary.result]

optional = widgets.SelectMultiple(
    options=optional_labor_force_catagory,
    #value=['Oranges'],
    layout=form_item_layout,
    rows=20,
    description='Optional',
    disabled=False
)
display(optional)

SelectMultiple(description='Optional', layout=Layout(align_items='stretch', border='solid 2px', display='flex'…

In [25]:
optional.value

('Computer and Mathematical', 'Arts, Design, Entertainment, Sports, and Media')

In [26]:
# Define Population function to choose the Weight for the Population

style = {'description_width': 'initial'}
Pop_weight = widgets.IntSlider(
    value=1,
    min=0,
    max=10,
    step=1,
    description='Population Weight',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)

display(Pop_weight)

IntSlider(value=1, continuous_update=False, description='Population Weight', max=10, style=SliderStyle(descrip…

In [27]:
Pop_weight.value


10

In [28]:
# Define Population function to choose the Weightage for the Labor Force


style = {'description_width': 'initial'}
Labor_weight = widgets.IntSlider(
    value=1,
    min=0,
    max=10,
    step=1,
    description='Labor Force Weight',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)

display(Labor_weight)

IntSlider(value=1, continuous_update=False, description='Labor Force Weight', max=10, style=SliderStyle(descri…

In [30]:
Labor_weight.value

4

In [31]:
# Define Population function to choose the Weightage for the Proximity to Airport

style = {'description_width': 'initial'}
if (Labor_weight.value == 0 and Pop_weight.value == 0):
    Airport_weight = widgets.IntSlider(
        value=1,
        min=1,
        max=10,
        step=1,
        description='Proximity to Airport Weight',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style=style
    )
else:
    Airport_weight = widgets.IntSlider(
        value=1,
        min=0,
        max=10,
        step=1,
        description='Proximity to Airport Weight',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style=style
    )


display(Airport_weight)

IntSlider(value=1, continuous_update=False, description='Proximity to Airport Weight', max=10, style=SliderSty…

In [32]:
Airport_weight.value

6